# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [23]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [24]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [25]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [26]:
# todo test
try:
    query_keyspace = """
    create keyspace if not exists songify
    with replication =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
    session.execute(query_keyspace)
except Exception as ex:
    print(ex)

#### Set Keyspace

In [27]:
try:
    session.set_keyspace('songify')
except Exception as e:
    print(e)

# Task 1

Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [28]:
"""
select -> artist name, song title, song lenght
from -> music app history
where -> session id 338 and iteminsession 4
"""

query = "create table if not exists music_app_history"
query += """
(
session_id int,
item_in_session int,
artist_name text,
song_title text,
song_length float,
primary key(session_id, item_in_session)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [29]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query = "insert into music_app_history (session_id, item_in_session, artist_name, song_title, song_length)"
        insert_query = insert_query + " values (%s, %s, %s, %s, %s) "
        try:
            session.execute(insert_query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))
        except Exception as ex:
            print(ex, line)

#### SELECT to verify 

In [30]:
select_query = """
select artist_name, song_title, song_length
from music_app_history
where session_id = 338
and item_in_session = 4
"""

try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)


Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


# Task 2

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [31]:
"""
select -> artist name, song, item in session, user name, latname
from -> 
where -> userid 10 sessionid 182
"""

second_query = "create table if not exists user_session"
second_query += """
(
session_id int,
item_in_session int,
artist_name text,
song_title text,
user_id int,
first_name text,
last_name text,
primary key((user_id, session_id), item_in_session)
)
"""
try:
    session.execute(second_query)
except Exception as e:
    print(e)                 

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query = "insert into user_session (session_id, item_in_session, artist_name, song_title, user_id, first_name, last_name)"
        insert_query = insert_query + " values (%s, %s, %s, %s, %s, %s, %s) "
        try:
            session.execute(insert_query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), int(line[10]), str(line[1]), str(line[4])))
        except Exception as ex:
            print(ex, line)

Select to verfiy it works

In [33]:
select_query = """
select artist_name, song_title, first_name, last_name
from user_session
where user_id = 10
and session_id = 182
"""

try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


# Task 3

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [34]:
"""
select -> firstname, lastname, user id (key), song_title
from -> 
where -> song -> text
"""

third_query = "create table if not exists users_by_song"
third_query += """
(
song_title text,
user_id int,
first_name text,
last_name text,
primary key(song_title, user_id)
)
"""
try:
    session.execute(third_query)
except Exception as e:
    print(e)               

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query = "insert into users_by_song (song_title, user_id, first_name, last_name)"
        insert_query = insert_query + " values (%s, %s, %s, %s) "
        try:
            session.execute(insert_query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))
        except Exception as ex:
            print(ex, line)

In [36]:
select_query = """
select first_name, last_name
from users_by_song
where song_title = 'All Hands Against His Own' ;
"""

try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [37]:
query = "drop table if exists music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists users_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()